# Sales Forecasting Regression – Walkthrough Notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error

%matplotlib inline


In [ ]:
df = pd.read_csv("sales_data.csv", parse_dates=['sale_date'])
df.head()

In [ ]:
df['day'] = df['sale_date'].dt.day
df['month'] = df['sale_date'].dt.month
df['year'] = df['sale_date'].dt.year
df['day_of_week'] = df['sale_date'].dt.dayofweek
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df.head()

In [ ]:
features = ['units_sold','discount_pct','marketing_spend','day','month','year','day_of_week','is_weekend']
X = df[features]
y = df['revenue']

tscv = TimeSeriesSplit(n_splits=5)
maes=[]; mses=[]
for tr,vl in tscv.split(X):
    Xtr, Xvl = X.iloc[tr], X.iloc[vl]
    ytr, yvl = y.iloc[tr], y.iloc[vl]
    sc = StandardScaler()
    Xtr_s = sc.fit_transform(Xtr)
    Xvl_s = sc.transform(Xvl)
    model = Ridge(alpha=1.0)
    model.fit(Xtr_s,ytr)
    p = model.predict(Xvl_s)
    maes.append(mean_absolute_error(yvl,p))
    mses.append(mean_squared_error(yvl,p))
print(np.mean(maes), np.mean(mses))

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model = Ridge(alpha=1.0)
model.fit(X_scaled,y)
df['predicted_revenue']=model.predict(X_scaled)
df.tail()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df['sale_date'],df['revenue'],label='Actual')
plt.plot(df['sale_date'],df['predicted_revenue'],label='Predicted')
plt.legend(); plt.show()